In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import datetime
from astropy.coordinates import EarthLocation, AltAz, get_sun, get_moon
from astropy.time import Time
from astropy import units as u
from astroquery.simbad import Simbad
from skyfield import api
from skyfield import almanac
from skyfield.toposlib import Topos
import pytz

In [ ]:
app = Flask(__name__)

In [4]:
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        latitude = request.form['latitude']
        longitude = request.form['longitude']
        start_date = request.form['start_date']
        end_date = request.form['end_date']
        start_time = request.form['start_time']
        end_time = request.form['end_time']
        objects_of_interest = request.form.getlist('objects_of_interest')
        custom_objects = request.form['custom_objects']
        min_altitude = request.form['min_altitude']
        max_altitude = request.form['max_altitude']
        min_azimuth = request.form['min_azimuth']
        max_azimuth = request.form['max_azimuth']

        # Load astronomical data
        load = api.Loader('skyfield-data')
        planets = load('de421.bsp')
        earth = planets['earth']
        ts = load.timescale()

        # Create location object
        observer_location = earth + Topos(latitude_degrees=float(latitude), longitude_degrees=float(longitude))

        # Define objects of interest
        celestial_objects = {
            'moon': planets['moon'],
            # Add more celestial objects here (planets, Messier objects, etc.)
        }

       # Find the observer's timezone based on the provided coordinates
        timezone = api.timezone_finder.timezone_at(lng=float(longitude), lat=float(latitude))

        # Convert date and time strings to datetime objects
        start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
        end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()
        start_time_dt = datetime.datetime.strptime(start_time, '%H:%M').time()
        end_time_dt = datetime.datetime.strptime(end_time, '%H:%M').time()

        # Combine the date and time objects
        start_local_datetime = datetime.datetime.combine(start_date_dt, start_time_dt)
        end_local_datetime = datetime.datetime.combine(end_date_dt, end_time_dt)

        # Convert the local observing times to UTC
        start_utc_datetime = timezone.localize(start_local_datetime).astimezone(pytz.UTC)
        end_utc_datetime = timezone.localize(end_local_datetime).astimezone(pytz.UTC)


        # Initialize the results list
        results = []

        # Iterate through the date range
        current_date = start_date_dt
        while current_date <= end_date_dt:
            start_datetime = datetime.datetime.combine(current_date, start_time_dt)
            end_datetime = datetime.datetime.combine(current_date, end_time_dt)

            if end_datetime <= start_datetime:
                end_datetime += datetime.timedelta(days=1)

            t0 = ts.utc(start_datetime.year, start_datetime.month, start_datetime.day, start_datetime.hour, start_datetime.minute, start_datetime.second)
            t1 = ts.utc(end_datetime.year, end_datetime.month, end_datetime.day, end_datetime.hour, end_datetime.minute, end_datetime.second)

            # Find the objects that are visible within the specified altitude and azimuth ranges
            for obj_name, obj in celestial_objects.items():
                if obj_name in objects_of_interest or obj_name in custom_objects:
                    t, is_rising = almanac.find_discrete(t0, t1, almanac.risings_and_settings(planets, obj, observer_location))

                    for ti, rising in zip(t, is_rising):
                        altitude, azimuth, _ = (observer_location - obj).at(ti).altaz()

                        if (rising
                            and min_altitude <= altitude.degrees <= max_altitude
                            and min_azimuth <= azimuth.degrees <= max_azimuth):

                            results.append({
                                'date': current_date,
                                'object': obj_name,
                                'altitude': round(altitude.degrees, 2),
                                'azimuth': round(azimuth.degrees, 2),
                            })

            current_date += datetime.timedelta(days=1)


        return render_template('index.html', results=results)
    return render_template('index.html')


In [ ]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
